In [0]:
!pip install simpy

In [0]:
# https://naps.com.mx/blog/simulacion-en-python-usando-simpy/
import simpy as smp
import random as rnd
import numpy as np
sem=30
atends=1
t_serv=2.5
t_llegadas=2.7
t_simul=240
cli=0
tet=0.0 # tiempo de espera total
dst=0.0 # duracion de servicio total
fin=0.0 # minuto en el que finaliza

def atender(cliente):
	global dst  #Para poder acceder a la variable dst declarada anteriormente
	t_servicio=-t_serv*np.log(rnd.random()) # Distribucion uniforme
	yield env.timeout(t_servicio) # deja correr el tiempo n minutos
	print(" \o/ Atención hecha %s en %.2f minutos" % (cliente,t_servicio))
	dst=dst+t_servicio # Acumula los tiempos de uso de la i

def cliente(env, nombre, personal ):
	global tet
	global fin
	llega=env.now # Guarda el minuto de llegada del cliente
	print("---> %s llegó al banco en el minuto %.2f" % (nombre, llega))
	with personal.request() as request: # Espera su turno
		yield request # Obtiene turno
		pasa=env.now # Guarda el minuto cuado comienza a ser atendido
		espera=pasa-llega # Calcula el tiempo que espero
		tet=tet+espera # Acumula los tiempos de espera
		print("**** %s pasa con el cajero en el minuto %.2f habiendo esperado %.2f" % (nombre, pasa, espera))
		yield env.process(atender(nombre)) # Invoca al proceso cortarate
		deja=env.now #Guarda el minuto en que termina el proceso cortar 
		print("<--- %s deja caja en minuto %.2f" % (nombre, deja))
		fin=deja # Conserva globalmente el ultimo minuto de la simulacion

def principal(env,personal):
    global cli
    while True: # Para tot_cli clientes
        llegada=-t_llegadas*np.log(rnd.random()) # Distribucion exponencial
        yield env.timeout(llegada)
        cli=cli+1
        env.process(cliente(env,'Cliente %d'%cli,personal))
        
rnd.seed(sem)  # Cualquier valor
env=smp.Environment() # Crea el objeto entorno de simulacion
personal=smp.Resource(env,atends) #Crea los recursos (servidores)
env.process(principal(env,personal)) #Invoca el proceso principal
env.run(until=t_simul) #Inicia la simulacion

print("\nIndicadores obtenidos: ")
print("\nLongitud promedio de la cola: %.2f" %(tet/fin))
print("Tiempo de espera promedio = %.2f" %(tet/cli))
print("Uso promedio de la instalacion = %.2f" %((dst/fin)/atends))

---> Cliente 1 llegó al banco en el minuto 1.67
**** Cliente 1 pasa con el cajero en el minuto 1.67 habiendo esperado 0.00
---> Cliente 2 llegó al banco en el minuto 5.02
---> Cliente 3 llegó al banco en el minuto 6.17
---> Cliente 4 llegó al banco en el minuto 10.38
 \o/ Atención hecha Cliente 1 en 8.76 minutos
<--- Cliente 1 deja caja en minuto 10.43
**** Cliente 2 pasa con el cajero en el minuto 10.43 habiendo esperado 5.41
 \o/ Atención hecha Cliente 2 en 2.31 minutos
<--- Cliente 2 deja caja en minuto 12.74
**** Cliente 3 pasa con el cajero en el minuto 12.74 habiendo esperado 6.57
 \o/ Atención hecha Cliente 3 en 1.11 minutos
<--- Cliente 3 deja caja en minuto 13.85
**** Cliente 4 pasa con el cajero en el minuto 13.85 habiendo esperado 3.47
 \o/ Atención hecha Cliente 4 en 0.03 minutos
<--- Cliente 4 deja caja en minuto 13.88
---> Cliente 5 llegó al banco en el minuto 14.05
**** Cliente 5 pasa con el cajero en el minuto 14.05 habiendo esperado 0.00
 \o/ Atención hecha Cliente 5 e